# <u>Practice Day 18</u>
Item-Based Collaborative Filtering(IB-CF)
***
Date: 18th November 2019<br>
Author: Samuel Natamihardja<br>
Company: Home Credit Indonesia

#### Importing Library

In [88]:
import pandas as pd #data wrangling
import numpy as np #calculation
import matplotlib.pyplot as plt #visualization
import seaborn as sns #visualization


from scipy.sparse import csr_matrix #prepare matrix

#Model
import surprise
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import Reader


#Model Evaluation
from sklearn import metrics
from sklearn.metrics import auc, roc_curve
from sklearn.metrics import accuracy_score

## Importing Dataset

In [89]:
#importing dataset
books = pd.read_csv('new_dataset/new_books.csv')
ratings = pd.read_csv('new_dataset/ratings.csv')

#### Copy Dataset

In [90]:
df_books = books[['book_id','original_publication_year','title','authors','tag_name','image_url']]

#### Handling Missing Value

In [91]:
df_books.isnull().sum()

book_id                       0
original_publication_year    21
title                         0
authors                       0
tag_name                      0
image_url                     0
dtype: int64

In [92]:
df_books.dropna(inplace = True)
df_books.isnull().sum()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


book_id                      0
original_publication_year    0
title                        0
authors                      0
tag_name                     0
image_url                    0
dtype: int64

In [93]:
df_ratings.isnull().sum()

book_id    0
user_id    0
rating     0
dtype: int64

In [94]:
df_ratings = df_ratings.astype(int)

In [95]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981756 entries, 0 to 981755
Data columns (total 3 columns):
book_id    981756 non-null int32
user_id    981756 non-null int32
rating     981756 non-null int32
dtypes: int32(3)
memory usage: 11.2 MB


### Final Dataset

In [96]:
df_books = df_books.astype({"original_publication_year": int})
df_books.head()

,book_id,original_publication_year,title,authors,tag_name,image_url
0,2767052,2008,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,young adult,https://images.gr-assets.com/books/1447303603m...
1,3,1997,Harry Potter and the Sorcerer's Stone (Harry P...,"J.K. Rowling, Mary GrandPré",fantasy,https://images.gr-assets.com/books/1474154022m...
2,41865,2005,"Twilight (Twilight, #1)",Stephenie Meyer,young adult,https://images.gr-assets.com/books/1361039443m...
3,2657,1960,To Kill a Mockingbird,Harper Lee,classics,https://images.gr-assets.com/books/1361975680m...
4,4671,1925,The Great Gatsby,F. Scott Fitzgerald,classics,https://images.gr-assets.com/books/1490528560m...


In [97]:
df_ratings = ratings.copy()
df_ratings.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


## Item-Based Collaborative Filtering(IB-CF)
With this method, item to item filtering tried to find items similarity.
Example question: "Users who liked this item also liked ..."

In [116]:
reader = Reader(rating_scale=(1, 5))

In [147]:
data = Dataset.load_from_df(df_ratings[["user_id", "book_id", "rating"]].head(200000), reader)

In [148]:
model_knn = KNNWithMeans(sim_options={"name":"msd","user_base":False},k=5)

In [149]:
data_train = data.build_full_trainset()

In [150]:
model_knn.fit(data_train)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [141]:
df_ratings.shape

(981756, 3)